# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [50]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [61]:
orders = pd.read_csv('Orders.csv')

orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [52]:
sub_1 = orders.groupby(['CustomerID']).agg({'amount_spent':'sum'}) ## aggregating amount_spent by unique customer 
sub_1


,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [53]:

sub_1['quantile'] = pd.qcut(sub_1['amount_spent'],20,labels = ['0%','5%','10%','15%','20%','25%','30%','35%','40%','45%','50%','55%','60%','65%',
'70%','75%','80%','85%','90%','95%']) ## creating 20 bins of equal size (not in range but in number of customers)

sub_1


,amount_spent,quantile
CustomerID,,
12346,77183.60,95%
12347,4310.00,90%
12348,1797.24,75%
12349,1757.55,75%
12350,334.40,25%
...,...,...
18280,180.60,10%
18281,80.82,0%
18282,178.05,10%


In [79]:
sub_2 = sub_1[sub_1['quantile'] == '95%'] ## filtering by the 95th quantile -->  ## there are the VIP customers



sub_2['CustomerID'] = sub_2.index


sub_2



/var/folders/v5/90hqz_xs4ys1hl5xzb98xyh40000gn/T/ipykernel_1229/2843733492.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_2['CustomerID'] = sub_2.index


,amount_spent,quantile,CustomerID
CustomerID,,,
12346,77183.60,95%,12346
12357,6207.67,95%,12357
12359,6372.58,95%,12359
12409,11072.67,95%,12409
12415,124914.53,95%,12415
...,...,...,...
18109,8052.97,95%,18109
18139,8438.34,95%,18139
18172,7561.68,95%,18172


In [78]:
sub_3 = sub_1[(sub_1['quantile'] == '75%') | (sub_1['quantile'] == '80%') | (sub_1['quantile'] == '85%')]

sub_3 ## preferred customers

,amount_spent,quantile
CustomerID,,
12348,1797.24,75%
12349,1757.55,75%
12352,2506.04,80%
12356,2811.43,85%
12360,2662.06,85%
...,...,...
18259,2338.60,80%
18260,2643.20,85%
18272,3078.58,85%


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [72]:
vip = pd.merge(sub_2, orders, how = 'inner', left_index= True, right_on='CustomerID') ## creating a table just with quantile 95%

vip['quantile'].unique() 

['95%']
Categories (20, object): ['0%' < '5%' < '10%' < '15%' ... '80%' < '85%' < '90%' < '95%']

In [73]:
vip['Country'].max() ## the UK has the most VIP customers

'United Kingdom'

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [82]:
vip_pref = pd.merge(sub_2, sub_3, how = 'outer', left_index= True, right_index =True) ## merging VIP and pref customers together into a single df
 
vip_pref

,amount_spent_x,quantile_x,CustomerID,amount_spent_y,quantile_y
CustomerID,,,,,
12346,77183.6,95%,12346.0,NaN,NaN
12348,NaN,NaN,NaN,1797.24,75%
12349,NaN,NaN,NaN,1757.55,75%
12352,NaN,NaN,NaN,2506.04,80%
12356,NaN,NaN,NaN,2811.43,85%
...,...,...,...,...,...
18259,NaN,NaN,NaN,2338.60,80%
18260,NaN,NaN,NaN,2643.20,85%
18272,NaN,NaN,NaN,3078.58,85%


In [83]:
vip_pref_country = pd.merge(vip_pref, orders, how = 'inner', left_index= True, right_on='CustomerID')

In [85]:
vip_pref_country['Country'].max() ## UK has the most VIP + Pref Customers combined

'United Kingdom'